In [1]:
import re
import pickle
import numpy as np
import pandas as pd
import datetime as dt

from tqdm import tqdm
from pathlib import Path
from joblib import Parallel, delayed

from ewma_var_calculator import EWMAVaRCalculator
from garch_var_calculator import GARCHVaRCalculator

In [2]:
simulations_directory = Path("llm_simulations")
data_directory = Path("datasets")

def read_original_asset_returns(asset: str):
    normalized_asset_name = "".join([f"[{char.lower()}{char.upper()}]" for char in asset])
    original_prices_file = list(data_directory.glob(f"{normalized_asset_name}.xlsx"))[0]
    original_prices = pd.read_excel(original_prices_file).set_index("Date")
    price_column = "Close" if asset != "stoxx" else "STOXX"
    log_returns = np.log(original_prices[price_column].pct_change() + 1)
    log_returns.fillna(log_returns.iloc[1], inplace=True)
    return log_returns

llmtime_assets = set()
asset_regex = re.compile(r".*?_([a-zA-Z0-9]+)_LLMTime(?:_| )GPT-3\.{1,2}5.*")
for simulations_file in simulations_directory.glob("*.xlsx"):
    llmtime_assets.add(asset_regex.match(simulations_file.stem).group(1))

llmtime_assets = list(llmtime_assets)

In [3]:
log_returns_dict = {asset: read_original_asset_returns(asset) for asset in llmtime_assets}

### EWMA

In [4]:
lambda_ = 0.95
alpha = [1 - 0.975, 1 - 0.99]
nu = 5 

In [5]:
ewma_VaR_results = {}

for asset in tqdm(llmtime_assets, desc="Running EWMA for all assets..."):
    ewma_var_calculator = EWMAVaRCalculator(log_returns_dict[asset], lambda_, alpha, 120, nu)
    ewma_var = ewma_var_calculator.EWMA_VaR()
    dcs_ewma_var = ewma_var_calculator.DCS_EWMA_VaR()
    sd_ewma_var = ewma_var_calculator.SD_EWMA_VaR()
    ewma_VaR_results[asset] = {"Norm": ewma_var, "DCS": dcs_ewma_var, "SD": sd_ewma_var}

Running EWMA for all assets...: 100%|██████████| 9/9 [00:10<00:00,  1.15s/it]


In [6]:
with Path("EWMA/EWMA_Simulations_120_window.pkl").open("wb") as f:
    pickle.dump(ewma_VaR_results, f)

In [7]:
ewma_VaR_results = {}

for asset in tqdm(llmtime_assets, desc="Running EWMA for all assets..."):
    ewma_var_calculator = EWMAVaRCalculator(log_returns_dict[asset], lambda_, alpha, 80, nu)
    ewma_var = ewma_var_calculator.EWMA_VaR()
    dcs_ewma_var = ewma_var_calculator.DCS_EWMA_VaR()
    sd_ewma_var = ewma_var_calculator.SD_EWMA_VaR()
    ewma_VaR_results[asset] = {"Norm": ewma_var, "DCS": dcs_ewma_var, "SD": sd_ewma_var}

Running EWMA for all assets...: 100%|██████████| 9/9 [00:08<00:00,  1.08it/s]


In [8]:
with Path("EWMA/EWMA_Simulations_80_window.pkl").open("wb") as f:
    pickle.dump(ewma_VaR_results, f)

### GARCH

In [4]:
def run_garch_model(asset, alpha, window, nu):
    var_model = GARCHVaRCalculator(log_returns_dict[asset], alpha, window, nu)
    garch_var = var_model.GARCH_VaR()
    garch_t_var = var_model.GARCH_t_VaR()
    gas_garch_var = var_model.GAS_GARCH_VaR()
    gas_garch_t_var = var_model.GAS_GARCH_t_VaR()
    
    return {"Norm": garch_var, "T": garch_t_var, "GAS": gas_garch_var, "GAS_T": gas_garch_t_var} 

In [5]:
alpha = [1 - 0.975, 1 - 0.99]
nu = 5

In [6]:
garch_parallel_results = Parallel(prefer="processes", n_jobs=2 * len(llmtime_assets), verbose=100)(
    delayed(run_garch_model)(asset, alpha, window, nu)
    for window in [120, 250]
    for asset in llmtime_assets
)

[Parallel(n_jobs=18)]: Using backend LokyBackend with 18 concurrent workers.
[Parallel(n_jobs=18)]: Done   1 tasks      | elapsed:  5.7min
[Parallel(n_jobs=18)]: Done   2 out of  18 | elapsed:  5.7min remaining: 45.9min
[Parallel(n_jobs=18)]: Done   3 out of  18 | elapsed:  5.8min remaining: 28.8min
[Parallel(n_jobs=18)]: Done   4 out of  18 | elapsed:  5.8min remaining: 20.1min
[Parallel(n_jobs=18)]: Done   5 out of  18 | elapsed:  5.8min remaining: 15.2min
[Parallel(n_jobs=18)]: Done   6 out of  18 | elapsed:  5.9min remaining: 11.7min
[Parallel(n_jobs=18)]: Done   7 out of  18 | elapsed:  5.9min remaining:  9.3min
[Parallel(n_jobs=18)]: Done   8 out of  18 | elapsed:  6.2min remaining:  7.7min
[Parallel(n_jobs=18)]: Done   9 out of  18 | elapsed:  8.0min remaining:  8.0min
[Parallel(n_jobs=18)]: Done  10 out of  18 | elapsed:  8.0min remaining:  6.4min
[Parallel(n_jobs=18)]: Done  11 out of  18 | elapsed:  8.1min remaining:  5.1min
[Parallel(n_jobs=18)]: Done  12 out of  18 | elapse

In [7]:
garch_window_120_results = {
    asset: garch_parallel_results[i]
    for i, asset in enumerate(llmtime_assets)
}

garch_window_250_results = {
    asset: garch_parallel_results[len(llmtime_assets) + i]
    for i, asset in enumerate(llmtime_assets)
}

In [8]:
with Path("GARCH/GARCH_Simulations_final_120_window.pkl").open("wb") as f:
    pickle.dump(garch_window_120_results, f)

with Path("GARCH/GARCH_Simulations_final_250_window.pkl").open("wb") as f:
    pickle.dump(garch_window_250_results, f)